In [38]:
import pandas as pd 
import matplotlib as pyplot

In [39]:
df = pd.read_csv("../rawdata/hololivecomments.csv")
del df['subreddit.1']
display(df.head(5))

,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
0,Hololive,ghnp0eh,ko1471,1609460553,SuperKalkorat,24,I agree what happened is ridiculously sad and ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
1,Hololive,ghnq0oo,ko1471,1609461129,Choppedcity,16,Nice try for trying karma farm from her.,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
2,Hololive,ghnomav,ko1471,1609460327,Karma110,11,We’re still going on about this?,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
3,Hololive,ghnsfkd,ko1471,1609462534,orz7db,7,I changed my flair yesterday to leave the bad ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
4,Hololive,ghntmas,ko1471,1609463238,DanteKir,6,It's called life. To move on and forge a path ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...


In [40]:
epoch_t = 1621132355
time_stamp = pd.to_datetime(epoch_t, unit='s')
# UTC (Coordinated Universal Time) is one of the well-known names of UTC+0 time zone which is 0h.
# By default, time series objects of pandas do not have an assigned time zone.
print("Regular time stamp in UTC:")
print(time_stamp)
print("\nConvert the said timestamp in to US/Pacific:")
print(time_stamp.tz_localize('UTC').tz_convert('US/Pacific'))
print("\nConvert the said timestamp in to Asia/Jakarta:")
print(time_stamp.tz_localize('UTC').tz_convert('Asia/Jakarta'))

Regular time stamp in UTC:
2021-05-16 02:32:35

Convert the said timestamp in to US/Pacific:
2021-05-15 19:32:35-07:00

Convert the said timestamp in to Asia/Jakarta:
2021-05-16 09:32:35+07:00


In [41]:
df['created_utc']=pd.to_datetime(df['created_utc'], unit='s')
display(df.head(5))
display(df.info())

,subreddit,id,submission_id,created_utc,author,score,body,parent_id,permalink
0,Hololive,ghnp0eh,ko1471,2021-01-01 00:22:33,SuperKalkorat,24,I agree what happened is ridiculously sad and ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
1,Hololive,ghnq0oo,ko1471,2021-01-01 00:32:09,Choppedcity,16,Nice try for trying karma farm from her.,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
2,Hololive,ghnomav,ko1471,2021-01-01 00:18:47,Karma110,11,We’re still going on about this?,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
3,Hololive,ghnsfkd,ko1471,2021-01-01 00:55:34,orz7db,7,I changed my flair yesterday to leave the bad ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...
4,Hololive,ghntmas,ko1471,2021-01-01 01:07:18,DanteKir,6,It's called life. To move on and forge a path ...,t3_ko1471,/r/Hololive/comments/ko1471/it_didnt_have_to_e...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1552210 entries, 0 to 1552209
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   subreddit      1552210 non-null  object        
 1   id             1552210 non-null  object        
 2   submission_id  1552210 non-null  object        
 3   created_utc    1552210 non-null  datetime64[ns]
 4   author         1496655 non-null  object        
 5   score          1552210 non-null  int64         
 6   body           1552208 non-null  object        
 7   parent_id      1552210 non-null  object        
 8   permalink      1552210 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 106.6+ MB


None